In [1]:
%env KIF_DEBUG=
%env KIF_RESOLVE_ENTITIES=1

env: KIF_DEBUG=
env: KIF_RESOLVE_ENTITIES=1


# SPARQL Mapping

In [2]:
from kif_lib import *
from kif_lib.vocabulary import pc, wd

### The mapping

In [3]:
from kif_lib.compiler.sparql.mapping import SPARQL_Mapping, register
from kif_lib.namespace import RDF, XSD
from kif_lib.namespace.semsci import CHEMINF, SIO
x, y, z = Variables('x', 'y', 'z')

In [22]:
class MyPubChemMapping(SPARQL_Mapping):

    @register([wd.canonical_SMILES(Item(x), String(y))], rank=Normal)
    def wd_canonical_SMILES(self, c, x, y):
        if isinstance(y, c.Query.Literal):
            y = c.literal(str(y), 'en')
        attr = c.bnode()
        c.q.triples()(
            (x, SIO.has_attribute, attr),
            (attr, RDF.type, CHEMINF.canonical_smiles_generated_by_OEChem),
            (attr, SIO.has_value, y))

    @register([wd.mass(Item(x), Quantity(y, wd.gram_per_mole))], rank=Normal)
    def wd_mass(self, c, x, y):
        if isinstance(y, c.Query.Literal):
            y = c.literal(y, datatype=XSD.float)
        attr = c.bnode()
        c.q.triples()(
            (x, SIO.has_attribute, attr),
            (attr, RDF.type, CHEMINF.molecular_weight_calculated_by_the_pubchem_software_library),
            (attr, SIO.has_value, y))
    
    @register([wd.instance_of(pc.Isotope_Atom_Count, wd.Wikidata_property_related_to_chemistry)], rank=Normal)
    def wd_instance_of_Isotope_Atom_Count(self, c):
        pass

    @register([pc.Isotope_Atom_Count(Item(x), Quantity(y))], rank=Normal)
    def wd_Isotope_Atom_Count(self, c, x, y):
        if isinstance(y, c.Query.Literal):
            y = c.literal(y, datatype=XSD.float)
        attr = c.bnode()
        c.q.triples()(
            (x, SIO.has_attribute, attr),
            (attr, RDF.type, CHEMINF.isotope_atom_count_generated_by_pubchem_software_library),
            (attr, SIO.has_value, y))

### Example queries

In [23]:
DATA = '''
@prefix cid: <http://rdf.ncbi.nlm.nih.gov/pubchem/compound/> .
@prefix semsci: <http://semanticscience.org/resource/> .

# instance of - type of a chemical entity
cid:CID241 semsci:SIO_000008 [
        a semsci:CHEMINF_000140 ;
        ] .

# mass - 78.11 gram per mole
cid:CID241 semsci:SIO_000008 [
        a semsci:CHEMINF_000334 ;
        semsci:SIO_000300 "78.11" ;
        ] .
'''

In [24]:
kb = Store('sparql', mapping=MyPubChemMapping(), data=DATA)

In [25]:
display(*kb.filter(annotated=True))

(**AnnotatedStatement** (**Property** [http://semanticscience.org/resource/CHEMINF_000372](http://semanticscience.org/resource/CHEMINF_000372)) (**ValueSnak** (**Property** [instance of](http://www.wikidata.org/entity/P31)) (**Item** [Wikidata property related to chemistry](http://www.wikidata.org/entity/Q21294996)))
- (**QualifierRecord**)
- (**ReferenceRecordSet**)
- **NormalRank**)

(**AnnotatedStatement** (**Item** [[6]annulene](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID241)) (**ValueSnak** (**Property** [mass](http://www.wikidata.org/entity/P2067)) (**Quantity** 78.11 (**Item** [gram per mole](http://www.wikidata.org/entity/Q28924752))))
- (**QualifierRecord**)
- (**ReferenceRecordSet**)
- **NormalRank**)